In [1]:
# This cell is removed with the tag: "remove-input"
# As such, it will not be shown in documentation

import warnings
warnings.filterwarnings('ignore')

(UserGuide_Tools_Basic_Select)=
# Select
*Selecting elements in a molecular system*

Elements selections is probably one of the most frequently tasks when we work with molecular systems. There are many circumstances under which we need to know list of elements acomplishing a certain condition. We probably need, for instance, to calculate de contact map between CA atoms from two chains, or to remove the solvent atoms or to know how many 'HIS' residues there are in a protein. All these conditions can be expresed as a sentence that a query over elements needs to match. Each library to work with molecular systems, molecular dynamics engine or molecular visualization software, have each own syntax you have to follow to write this sentence. See for instance different examples in the tools [MDTraj](https://www.mdtraj.org/1.9.7/atom_selection.html), [PyTraj](https://amber-md.github.io/pytraj/latest/atom_mask_selection.html?highlight=select), [MDAnalysis](https://docs.mdanalysis.org/stable/documentation_pages/selections.html), [NGLView](http://nglviewer.org/ngl/api/manual/usage/selection-language.html).

The basic module of MolSysMT includes the function `molsysmt.basic.select` to perform elements selection on molecular systems of any supported form.

## MolSysMT selection syntax

Although you can use the function {func}`molsysmt.basic.select` with your preferred syntax (see XXX), MolSysMT has its own selection syntax based on the attributes of the elements as atoms, groups, molecules, etc. Let's load a molecular system to explain the logic behind this syntax:

In [2]:
import molsysmt as msm

In [3]:
molecular_system = msm.convert('1TCD', to_form='molsysmt.MolSys')

A molecular system encoded as the native form 'molsysmt.MolSys' has a pandas DataFrame with the atoms breakdown:

In [4]:
molecular_system.topology.atoms_dataframe

,atom_index,atom_name,atom_id,atom_type,group_index,group_name,group_id,group_type,component_index,component_name,...,molecule_type,entity_index,entity_name,entity_id,entity_type,occupancy,alternate_location,b_factor,formal_charge,partial_charge
0,0,N,1,N,0,LYS,4,aminoacid,0,0,...,protein,0,Triosephosphate isomerase,0,protein,1.0,None,0.3226,0.0,None
1,1,CA,2,C,0,LYS,4,aminoacid,0,0,...,protein,0,Triosephosphate isomerase,0,protein,1.0,None,0.3328,0.0,None
2,2,C,3,C,0,LYS,4,aminoacid,0,0,...,protein,0,Triosephosphate isomerase,0,protein,1.0,None,0.3253,0.0,None
3,3,O,4,O,0,LYS,4,aminoacid,0,0,...,protein,0,Triosephosphate isomerase,0,protein,1.0,None,0.3216,0.0,None
4,4,CB,5,C,0,LYS,4,aminoacid,0,0,...,protein,0,Triosephosphate isomerase,0,protein,1.0,None,0.2535,0.0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3978,3978,O,3979,O,657,HOH,339,water,162,162,...,water,1,water,1,water,1.0,None,0.4365,0.0,None
3979,3979,O,3980,O,658,HOH,340,water,163,163,...,water,1,water,1,water,1.0,None,0.3323,0.0,None
3980,3980,O,3981,O,659,HOH,341,water,164,164,...,water,1,water,1,water,1.0,None,0.3706,0.0,None
3981,3981,O,3982,O,660,HOH,342,water,165,165,...,water,1,water,1,water,1.0,None,0.3207,0.0,None


As you can see, the column names are the fundamental attributes of the molecular system atoms:

In [5]:
print(molecular_system.topology.atoms_dataframe.columns)

Index(['atom_index', 'atom_name', 'atom_id', 'atom_type', 'group_index',
       'group_name', 'group_id', 'group_type', 'component_index',
       'component_name', 'component_id', 'component_type', 'chain_index',
       'chain_name', 'chain_id', 'chain_type', 'molecule_index',
       'molecule_name', 'molecule_id', 'molecule_type', 'entity_index',
       'entity_name', 'entity_id', 'entity_type', 'occupancy',
       'alternate_location', 'b_factor', 'formal_charge', 'partial_charge'],
      dtype='object')


These attributes define the keywords of the attributes:

<center>

| Word | Attribute | Accepted Synonyms | Meaning |
|---|---|---|---|
| atom_index | atom_index | "atom_indices", "index" or "indices" when `element=atom`| Atom index/ices |
| atom_name | atom_name | "atom_names", "name" or "names" when `element=atom`| Atom name/s |
| atom_id | atom_id | "atom_ids", "id" or "ids" when `element=atom`| Atom id/s |
| atom_type | atom_type | "atom_types", "type" or "types" when `element=atom`| Atom type/s |
| group_index | group_index | "group_indices", "index" or "indices" when `element=group`| Group index/ices |
| group_name | group_name | "atom_names", "name" or "names" when `element=group`| Group name/s |
| group_id | group_id | "group_ids", "id" or "ids" when `element=group`| Group id/s |
| group_type | group_type | "group_types", "type" or "types" when `element=group`| Group type/s |
| component_index | component_index | "component_indices", "index" or "indices" when `element=component`| Component index/ices |
| component_name | component_name | "component_names", "name" or "names" when `element=component`| Component name/s |
| component_id | component_id | "component_ids", "id" or "ids" when `element=component`| Component id/s |
| component_type | component_type | "component_types", "type" or "types" when `element=component`| Component type/s |
| chain_index | chain_index | "chain_indices", "index" or "indices" when `element=chain`| Chain index/ices |
| chain_name | chain_name | "chain_names", "name" or "names" when `element=chain`| Chain name/s |
| chain_id | chain_id | "chain_ids", "id" or "ids" when `element=chain`| Chain id/s |
| chain_type | chain_type | "chain_types", "type" or "types" when `element=chain`| Chain type/s |
| molecule_index | molecule_index | "molecule_indices", "index" or "indices" when `element=molecule`| Molecule index/ices |
| molecule_name | molecule_name | "molecule_names", "name" or "names" when `element=molecule`| Molecule name/s |
| molecule_id | molecule_id | "molecule_ids", "id" or "ids" when `element=molecule`| Molecule id/s |
| molecule_type | molecule_type | "molecule_types", "type" or "types" when `element=molecule`| Molecule type/s |  
| entity_index | entity_index | "molecule_indices", "index" or "indices" when `element=molecule`| Entity index/ices |
| entity_name | entity_name | "entity_names", "name" or "names" when `element=entity`| Entity name/s |
| entity_id | entity_id | "entity_ids", "id" or "ids" when `element=molecule`| Entity id/s |
| entity_type | entity_type | "entity_types", "type" or "types" when `element=entity`| Entity type/s |  
| occupancy | occupancy | "occupancies" | Atom occupancy |
| alternate_location | alternate_location | "alternate_locations" | Atom alternate location id |
| b_factor | b_factor | "b_factors" | B-factors or Debye-Waller factors |
| formal_charge | formal_charge | "formal_charges" | Formal charge |
| partial_charge | partial_charge | "partial_charges" | Partial charge |
</center>

The syntax proposed by Pandas to perform queries in a 'pandas.DataFrame' is the base of the MolSysMT selection procedure. The boolean syntax of Pandas includes the following words and symbols:

<center>

| Word | Symbol | Meaning |
|---|---|---|
| and | & | and |
| or | \| | or |
| not | ~ | not |
| in | | in |
|  | == | equal |
|  | != | not equal |
|  | < | less than |
|  | <= | less or equal than |
|  | > | greater than |
|  | >= | greater or equal than |

</center>



## Some examples of atoms selections
The following are some examples where a list of atoms is obtained matching some selection criteria based on atoms attributes:

In [6]:
# Atoms with name CA or CB
msm.select(molecular_system, element='atom', selection='atom_name in ["CA","CB"] and atom_id<20')

array([ 1,  4, 10, 13, 17])

```{admonition} Tip
:class: tip
All functions defined in the {ref}`molsysmt.basic <API basic>` module can be invoked also from the main level of the library. Hence, {func}`molsysmt.select` is the same function as {func}`molsysmt.basic.selection`.
```

In [7]:
# Heavy atoms
msm.select(molecular_system, 'not atom_type=="H"')

array([   0,    1,    2, ..., 3980, 3981, 3982])

```{admonition} Tip
:class: tip
By default, the input argument `element` takes the value "atom" in {func}`molsysmt.basic.selection`.
```

In [8]:
# Atoms of type C not named CA
msm.select(molecular_system, 'atom_type=="C" and not atom_name=="CA"')

array([   2,    4,    5, ..., 3813, 3814, 3815])

In [9]:
# Atoms not named CA, CB or C
msm.select(molecular_system, 'atom_name!=["CA","CB","C"]')

array([   0,    3,    5, ..., 3980, 3981, 3982])

Atoms can be selected using attributes of other the other elements in the hierarchical organization of the molecular system: 'group', 'component', 'molecule', 'chain', 'entity'. You can find further information of these elements in XXX. These are some examples of selection sentences including other criteria than atoms attributes:

In [10]:
# Atoms belonging to molecules of type water.
msm.select(molecular_system, 'molecule_type=="water"')

array([3818, 3819, 3820, 3821, 3822, 3823, 3824, 3825, 3826, 3827, 3828,
       3829, 3830, 3831, 3832, 3833, 3834, 3835, 3836, 3837, 3838, 3839,
       3840, 3841, 3842, 3843, 3844, 3845, 3846, 3847, 3848, 3849, 3850,
       3851, 3852, 3853, 3854, 3855, 3856, 3857, 3858, 3859, 3860, 3861,
       3862, 3863, 3864, 3865, 3866, 3867, 3868, 3869, 3870, 3871, 3872,
       3873, 3874, 3875, 3876, 3877, 3878, 3879, 3880, 3881, 3882, 3883,
       3884, 3885, 3886, 3887, 3888, 3889, 3890, 3891, 3892, 3893, 3894,
       3895, 3896, 3897, 3898, 3899, 3900, 3901, 3902, 3903, 3904, 3905,
       3906, 3907, 3908, 3909, 3910, 3911, 3912, 3913, 3914, 3915, 3916,
       3917, 3918, 3919, 3920, 3921, 3922, 3923, 3924, 3925, 3926, 3927,
       3928, 3929, 3930, 3931, 3932, 3933, 3934, 3935, 3936, 3937, 3938,
       3939, 3940, 3941, 3942, 3943, 3944, 3945, 3946, 3947, 3948, 3949,
       3950, 3951, 3952, 3953, 3954, 3955, 3956, 3957, 3958, 3959, 3960,
       3961, 3962, 3963, 3964, 3965, 3966, 3967, 39

In [11]:
# Heavy atoms belonging to molecules of type protein.
msm.select(molecular_system, 'molecule_type=="protein" and atom_type!="H" and group_index==3')

array([25, 26, 27, 28, 29, 30, 31])

In [12]:
# Atoms belonging to residues named GLY, ALA or VAL in chain id A.
msm.select(molecular_system, 'group_name==["GLY","ALA","VAL"] and chain_id=="A"') 

array([  40,   41,   42,   43,   44,   45,   46,   47,   48,   49,   50,
         51,   52,   53,   54,  100,  101,  102,  103,  141,  142,  143,
        144,  145,  146,  147,  203,  204,  205,  206,  207,  208,  209,
        210,  211,  212,  257,  258,  259,  260,  261,  262,  263,  279,
        280,  281,  282,  283,  284,  285,  286,  287,  288,  289,  290,
        291,  292,  293,  294,  295,  296,  297,  380,  381,  382,  383,
        384,  463,  464,  465,  466,  467,  468,  469,  470,  471,  472,
        490,  491,  492,  493,  494,  527,  528,  529,  530,  531,  532,
        533,  534,  535,  554,  555,  556,  557,  567,  568,  569,  570,
        571,  572,  573,  642,  643,  644,  645,  674,  675,  676,  677,
        678,  679,  680,  681,  682,  683,  684,  685,  686,  687,  696,
        697,  698,  699,  779,  780,  781,  782,  824,  825,  826,  827,
        828,  829,  830,  831,  832,  833,  834,  835,  854,  855,  856,
        857,  858,  859,  860,  861,  862,  863,  8

The selection argument accepts also lists of indices, which are **always** intepreted as atoms indices.

In [13]:
msm.select(molecular_system, selection=[0,1,2])

array([0, 1, 2])

## Selecting elements other than atoms

The selection method of MolSysMT can also return other elements indices than atoms. As many methods in this library, {func}`molsysmt.basic.select()` has an input argument named `element` to select the elements nature of the output list of indices. Lets see some examples:

In [14]:
# Groups with name "ALA"
msm.select(molecular_system,  element='group', selection='group_name=="ALA"')

array([  5,   6,   7,  27,  28,  39,  50,  60,  61,  64,  70, 107, 111,
       113, 115, 116, 133, 137, 138, 145, 146, 148, 155, 162, 168, 175,
       180, 200, 201, 213, 216, 233, 245, 254, 255, 256, 276, 277, 288,
       299, 309, 310, 313, 319, 356, 360, 362, 364, 365, 382, 386, 387,
       394, 395, 397, 404, 411, 417, 424, 429, 449, 450, 462, 465, 482,
       494])

In [15]:
# Groups of atoms index 34, 44 or 64
msm.select(molecular_system, element='group', selection='atom_index==[34,44,64]')

array([4, 5, 9])

In [16]:
# Groups belonging to chain id A or C and molecule of type anything but water
msm.select(molecular_system, element='group', selection='chain_id in ["A", "C"] and molecule_type!="water"')

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

In [17]:
# Molecules of type water
msm.select(molecular_system, 'molecule_type=="water"', element='molecule')

array([  2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,  14,
        15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,
        28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,
        41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,
        54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,
        67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,  79,
        80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,  92,
        93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104, 105,
       106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118,
       119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131,
       132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144,
       145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157,
       158, 159, 160, 161, 162, 163, 164, 165, 166])

In [18]:
# Chains with molecules of type water
msm.select(molecular_system, 'molecule_type=="water"', element='chain')

array([2, 3])

In [19]:
# Bonds in group index 5
msm.select(molecular_system, 'group_index==5', element='bond')

array([37, 38, 39, 40])

When selection takes a list of integers as value, this is **always** intepreted as a list of atom indices:

In [20]:
msm.select(molecular_system, element='group', selection=[0,1,2,3,4,5,6,7,8,9,10,11])

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

In [21]:
msm.select(molecular_system, element='molecule', selection=[3900, 3910, 3920])

array([3900, 3910, 3920])

## Including external variables in the selection sentence

Pandas query method allows the use of external variables in the logical sentence. To include them, variables names have to be preceded by the character '@'. Lets illustrate its use with some examples:

In [22]:
# Atoms in groups with indices 10, 11 or 12.
indices=[10,11,12]
msm.select(molecular_system, 'group_index==@indices')

array([77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93,
       94, 95, 96, 97, 98, 99])

In [23]:
# Atoms named CA, C, O or N in groups with indices 10 to 29.
indices=list(range(10,30))
atoms=["CA", "C", "O", "N"]
msm.select(molecular_system, 'atom_name==@atoms & atom_index==@indices') 

array([10, 11, 12, 16, 17, 18, 19, 25, 26, 27, 28])

In [24]:
# Groups with indices equal to 0, 100 or 200
indices=[0,100,200]
msm.select(molecular_system, element='group', selection='group_index==@indices')

array([  0, 100, 200])

## Including mask filters in the selection

Although including masks is not really necessary, {func}`molsysmt.basic.select` has an optional input argument to do so:

In [25]:
# Atoms named C with atom index in range 10 to 29
indices=list(range(10,30))
msm.select(molecular_system, 'atom_name=="C"', mask=indices)

array([11, 18, 27])

In [26]:
# Atoms with index from 0 to 4 and from 0 to 2
msm.select(molecular_system, 'atom_index in [0,1,2,3,4]', mask=[0,1,2], element='atom')

array([0, 1, 2])

In [27]:
# Groups with index from 0 to 4 and from 0 to 2
msm.select(molecular_system, 'group_index in [0,1,2,3,4]', mask=[0,1,2], element='group')

array([0, 1, 2])

The use of masks can always be avoid using the logical sentence:

In [28]:
# Atoms named C with atom index in range 10 to 29
indices=list(range(10,30))
msm.select(molecular_system, 'atom_name=="C" and atom_index in @indices')

array([11, 18, 27])

## Selecting elements "within a distance of"

A selection of elements within a certain distance of a set of elements can be obtained using the string `within ... of`:

In [29]:
msm.select(molecular_system, 'chain_id=="A" within 0.3 nm of chain_id=="B"')

array([ 89, 480, 527, 547, 550, 552, 554, 566, 723, 734])

In [30]:
msm.select(molecular_system, '(atom_name=="N" and chain_id=="A") within 3 angstroms of (atom_type=="O" and molecule_type=="water")')

array([ 119,  213,  473,  531,  654,  696,  799, 1049])

In [31]:
msm.select(molecular_system, '(atom_name=="CA" and chain_id=="A") within 0.5 nm of (atom_name=="CA" and chain_id=="B")',
          element='group')

array([10, 42, 62, 72, 73])

The string "not within ... of ..." can also be used:

In [32]:
msm.select(molecular_system, 'chain_id=="A" not within 7.8 nanometers of chain_id=="B"')

array([1521, 1522, 1723, 1724])

And distances can be interpreted with or without periodic boundary conditions:

In [33]:
msm.select(molecular_system, 'chain_id=="A" within 0.3 nm without pbc of chain_id=="B"')

array([ 89, 480, 527, 547, 550, 552, 554, 566, 723, 734])

In [34]:
msm.select(molecular_system, 'chain_id=="A" within 0.3 nm with pbc of chain_id=="B"')

array([ 89, 480, 527, 547, 550, 552, 554, 566, 723, 734])

## Selecting atoms "bonded to ..."

Atoms bonded to specific atoms can also be selected with `bonded to`:

In [35]:
msm.select(molecular_system, 'atom_name=="N" bonded to atom_type=="C"')

array([   0,    9,   16,   25,   32,   40,   45,   50,   55,   63,   77,
         86,   92,  100,  104,  110,  119,  125,  133,  141,  148,  155,
        163,  171,  180,  187,  195,  203,  208,  213,  220,  231,  239,
        249,  257,  264,  273,  279,  286,  293,  298,  305,  312,  323,
        331,  341,  349,  356,  364,  371,  380,  385,  396,  404,  411,
        419,  426,  435,  446,  455,  463,  468,  473,  482,  490,  495,
        503,  510,  521,  527,  531,  536,  547,  554,  558,  567,  574,
        580,  588,  597,  605,  613,  622,  630,  642,  646,  654,  660,
        674,  681,  688,  696,  700,  710,  716,  725,  736,  747,  755,
        767,  779,  783,  792,  799,  807,  816,  824,  831,  836,  845,
        854,  861,  866,  875,  880,  886,  891,  896,  900,  911,  921,
        928,  936,  943,  949,  956,  960,  969,  976,  984,  993, 1002,
       1013, 1022, 1027, 1031, 1042, 1049, 1054, 1059, 1066, 1073, 1081,
       1088, 1097, 1105, 1110, 1115, 1122, 1127, 11

In [36]:
msm.select(molecular_system, '(atom_type=="O" and chain_id=="A") bonded to (atom_type=="C" and chain_id=="A")')

array([   3,   12,   19,   23,   28,   35,   43,   48,   53,   58,   61,
         66,   80,   89,   95,   98,  103,  107,  109,  113,  117,  118,
        122,  124,  128,  136,  144,  151,  158,  166,  174,  178,  179,
        183,  185,  190,  198,  201,  206,  211,  216,  218,  223,  234,
        237,  238,  242,  252,  255,  256,  260,  267,  271,  276,  282,
        289,  296,  301,  308,  310,  315,  326,  334,  344,  352,  359,
        367,  369,  374,  383,  388,  399,  407,  409,  414,  417,  422,
        429,  438,  449,  453,  458,  466,  471,  476,  480,  485,  488,
        493,  498,  506,  508,  513,  524,  526,  530,  534,  539,  550,
        552,  557,  561,  565,  566,  570,  577,  579,  583,  591,  595,
        600,  608,  616,  625,  628,  629,  633,  641,  645,  649,  657,
        659,  663,  677,  684,  691,  699,  703,  713,  715,  719,  723,
        724,  728,  739,  750,  758,  766,  770,  778,  782,  786,  790,
        791,  795,  797,  802,  805,  810,  814,  8

The string "not bonded to" can also be used:

In [37]:
msm.select(molecular_system, '(all not bonded to atom_type==["H","N","C","O"]) and molecule_type=="protein"')

array([ 363, 1714, 2275, 3626])

And both, `within .. of` and `bonded to`, can be mixed in the same selection sentence:

In [38]:
msm.select(molecular_system, '((atom_name=="N" and chain_id=="A") bonded to atom_type=="C") within 3 angstroms of (atom_type=="O" and molecule_type=="water")')

array([ 119,  213,  473,  531,  654,  696,  799, 1049])

## Other syntaxes supported

There is no need for the user to learn the native syntax. Other syntaxes such as the used by MDTraj or NGLView are also supported by MolSysMT:

In [39]:
msm.select(molecular_system, selection='(name =~ "C[A-B]") and (resid 1 to 3)', syntax='MDTraj')

array([10, 13, 17, 20, 26, 29])

## Translation between different syntaxes

MolSysMT is prepared to easily interact with other tools. The main goal of this library is providing with a set of pipes and joins to set up your workflows, keeping simple the integration of other tools. But different tools have different selection syntax. Learning how to use the selection syntax of MDTraj, ParmEd or NGLview is something very useful. Those are tools that we all use frequently in our labs. But it happens that we forget soon the rules of each tool. To keep a unique selection syntax in your projects, MolSysMT includes the input argument `to_syntax` in the method `molsysmt.select()`. Lets illustrate some examples:

In [40]:
msm.select(molecular_system, selection='group_index==[3,4,5]', to_syntax='NGLView')

'@25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44'

In [41]:
msm.select(molecular_system, selection='group_index==[3,4,5]', to_syntax='MDTraj')

'index 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44'

The output string can be obtained, if the selection is done over other elementted elements, as a sequence of groups or chains:

In [42]:
msm.select(molecular_system, element='group', selection='group_index==[3,4,5]', to_syntax='NGLView')

'7:A 8:A 9:A'

In [43]:
msm.select(molecular_system, element='group', selection='group_index==[3,4,5]', to_syntax='MDTraj')

'resid 3 4 5'

```{admonition} See also
:class: attention
{func}`molsysmt.basic.select`, {func}`molsysmt.basic.convert`, {func}`molsysmt.basic.info`
```